# CSE 572: Lab 18

In this lab, you will practice implementing anomaly detection (also known as outlier detection) techniques for a real-world dataset. Anomaly detection is the task of identifying instances whose characteristics differ significantly from the rest of the data. You should refer to the lecture slides and Chapter 9 of the "Introduction to Data Mining" book to understand some of the concepts in this tutorial. 

Acknowledgment: This notebook was adapted from Introduction to Data Mining, 2nd Edition. Tan, Steinbach, Karpatne, Kumar.

To execute and make changes to this notebook, click File > Save a copy to save your own version in your Google Drive or Github. Read the step-by-step instructions below carefully. To execute the code, click on each cell below and press the SHIFT-ENTER keys simultaneously or by clicking the Play button. 

When you finish executing all code/exercises, save your notebook then download a copy (.ipynb file). Submit the following **three** things:
1. a link to your Colab notebook,
2. the .ipynb file, and
3. a pdf of the executed notebook on Canvas.

To generate a pdf of the notebook, click File > Print > Save as PDF.

## Model-based statistical approaches

This approach assumes that the majority of the data instances are governed by some well-known probability distribution, e.g., Binomial or Gaussian distribution. Anomalies can then be detected as observations that do not fit the overall distribution of the data. 

In this example, our goal is to detect anomalous changes in the daily closing prices of various stocks. The input data *stocks.csv* contains the historical closing prices of stocks for 3 large corporations (Microsoft, Ford Motor Company, and Bank of America). 

In [ ]:
import pandas as pd

stocks = pd.read_csv('https://docs.google.com/uc?export=download&id=1UqHZmlfSoPDcZlTIr2TB6OadBhni9Kbv', header='infer')
stocks

stocks.index = stocks['Date']
stocks = stocks.drop(['Date'], axis=1)
stocks.head()

We can compute the percentage of changes in the daily closing price of each stock as follows:
\begin{equation}
\Delta(t) = 100 \times \frac{x_t - x_{t-1}}{x_{t-1}} 
\end{equation}

where $x_t$ denotes the price of a stock on day $t$ and $x_{t-1}$ denotes the price on its previous day, $t-1$.

In [ ]:
import numpy as np

N, d = stocks.shape

delta = pd.DataFrame(100*np.divide(stocks.iloc[1:,:].values-stocks.iloc[:N-1,:].values, stocks.iloc[:N-1,:].values),
                     columns=stocks.columns, 
                     index=stocks.iloc[1:].index)

delta.head()

Plot the daily change in price as a line for each of the companies.

In [ ]:
# MSFT - YOUR CODE HERE

In [ ]:
# F - YOUR CODE HERE

In [ ]:
# BAC - YOUR CODE HERE

We can also plot the *distribution* of the percentage daily changes in stock price for all three features (treated as attributes) as a 3D scatter plot.

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(8,5))
ax = plt.axes(projection='3d')
ax.scatter(delta.MSFT, delta.F, delta.BAC)
ax.set_xlabel('Microsoft')
ax.set_ylabel('Ford')
ax.set_zlabel('Bank of America')
plt.show()

Assuming the data follow a multivariate Gaussian distribution, we can compute the mean and covariance matrix of the 3-dimensional data. Compute and print these in the cell below. Name your variables `mean` and `cov`.

In [ ]:
# YOUR CODE HERE

To determine the anomalous trading days, we can compute the Mahalanobis distance between the percentage of price change on each day against the mean percentage of price change.

\begin{equation}
\textrm{Mahalanobis}(x) = (x - \mu_x) \Sigma^{-1}(x - \mu_x)^T
\end{equation}

where $x$ is assumed to be a row vector, $\mu_x$ is the feature-wise mean of $x$, and $\Sigma^{-1}$ is the inverted covariance matrix of $x$.

In [ ]:
from numpy.linalg import inv

# Invert the covariance matrix
cov_inv = np.linalg.inv(cov.to_numpy())

# Function to compute the Mahalanobis distance for one sample
def mahalanobis(row):
    sub = row - mean
    return np.matmul(sub, cov_inv).dot(sub)   

# Compute the Mahalanobis distance for every sample
anomaly_scores = np.apply_along_axis(mahalanobis, axis=1, arr=delta)

# Result is one score for each sample
anomaly_scores.shape

Plot the dataset as a 3d scatter plot again, but this time color the points by their anomaly score. Use `cmap=jet` for the colormap.

In [ ]:
# YOUR CODE HERE

The top anomalies are shown as brownish-orange points in the figure above. To see which samples the top anomalies correspond to, create a new dataframe named `result` that appends the anomaly scores as a new column to `delta`. 

In [ ]:
# YOUR CODE HERE

Now we can print the N samples (here, 10 samples) with the largest anomaly scores using the `nlargest()` function.

In [ ]:
result.nlargest(10, 'Anomaly score')

See the Wikipedia article for the global financial crisis for context on what happened during these dates: https://en.wikipedia.org/wiki/Global_financial_crisis_in_October_2008

**Question 1: We can see that the top anomaly corresponds to the sample from October 13, 2008, which represents the change between October 12-13, 2008. According to the Wikipedia article, what happened on this date?**

**Answer:**

YOUR ANSWER HERE

The plot below visualizes what happened to the stock prices during the first couple weeks of October 2008.

In [ ]:
fig, ax = plt.subplots(1)

delta[440:452].plot(ax=ax)
ax.set_ylabel('Percent Change')

The second largest anomaly corresponds to the sample from November 26, 2008. We plot the time series around that date below.

In [ ]:
fig, ax = plt.subplots(1)

delta[475:488].plot(ax=ax)
ax.set_ylabel('Percent Change')

## Distance-based (model free) approaches

This is a model-free anomaly detection approach as it does not require constructing an explicit model of the normal class to determine the anomaly score of data instances. The example code shown below employs the k-nearest neighbor approach to calculate anomaly score. 

Specifically, a normal instance is expected to have a small distance to its k-th nearest neighbor whereas an anomaly is likely to have a large distance to its k-th nearest neighbor. 

In the example below, we apply the distance-based approach with k=4 to identify the anomalous trading days from the stock market data described in the previous section.

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance

k = 4
nbrs = NearestNeighbors(n_neighbors=k, metric=distance.euclidean).fit(delta.to_numpy())
distances, indices = nbrs.kneighbors(delta.to_numpy())

anomaly_score = distances[:, k-1]

Plot the dataset as a 3d scatter plot again with points colored by their anomaly score computed using the distance to k-nearest neighbors approach. Use `cmap=jet` for the colormap.

In [ ]:
# YOUR CODE HERE

The results are slightly different than the results from the previous section because we have used Euclidean distance (instead of Mahalanobis distance) to detect the anomalies.

We can examine the dates associated with the top-10 highest anomaly scores as follows. 

In [ ]:
anom = pd.DataFrame(anomaly_score, index=delta.index, columns=['Anomaly score'])
result = pd.concat((delta, anom), axis=1)
result.nlargest(10, 'Anomaly score')